In [9]:
import sys
sys.path.append("../../")

from itertools import product

from PyTorchCML import losses, models, samplers, regularizers, evaluators, trainers
import torch
import random as rd
from torch import nn, optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from tqdm import tqdm

In [2]:
def svd_init(X, dim):
    """
    Args :
        X : csr_matrix which element is 0 or 1.
        dim : number of dimention
    """
    svd = TruncatedSVD(n_components=10)
    U_ = svd.fit_transform(X)
    V_ = svd.components_

    s = (U_.sum(axis=1).mean() + V_.sum(axis=0).mean()) / 2
    U = 2 ** 0.5 * U_ - (1 / n_dim) ** 0.5 * s * np.ones_like(U_)
    V = 2 ** 0.5 * V_ + (1 / n_dim) ** 0.5 / s * np.ones_like(V_)
    ub = -(2 / n_dim) ** 0.5 * U_.sum(axis=1) / s
    vb = (2 / n_dim) ** 0.5 * V_.sum(axis=0) * s

    return U, V, ub, vb

In [3]:
# download movielens dataset
movielens = pd.read_csv(
  '/opt/ml/input/data/train/train_ratings.csv', 
    sep=','
)

In [4]:
user_mapping = {idx: i for i, idx in enumerate(movielens.user.unique())}
item_mapping = {idx: i for i, idx in enumerate(movielens.item.unique())}

item_id = [item_mapping[idx] for idx in movielens['item']]
user_id = [user_mapping[idx] for idx in movielens['user']]

ml = pd.DataFrame({'user_id': user_id, 'item_id': item_id})

train, test = train_test_split(ml)

In [5]:
ml.nunique()

user_id    31360
item_id     6807
dtype: int64

In [6]:
test.head()

,user_id,item_id
4919800,29940,3606
1799008,10880,373
1361409,8197,2555
3797583,23097,1948
765926,4630,2974


In [7]:
test_items={}
for i, group in test.groupby('user_id'):
    test_items[i] = group.item_id.values

In [12]:
neg_pools = {}

for u in tqdm(test_items.keys()):
    neg_items = list(set(range(6807)) - set(test_items[u]))
    pools = [rd.choice(neg_items) for _ in range(600)]
    neg_pools[u] = pools

100%|██████████| 31360/31360 [00:25<00:00, 1211.32it/s]


In [13]:
for k, v in tqdm(neg_pools.items()):
    df_neg = pd.DataFrame({'user_id': np.repeat(k,600), 'item_id': v})
    test = pd.concat([test, df_neg])

test.to_csv('test.csv')

100%|██████████| 31360/31360 [1:10:30<00:00,  7.41it/s]


In [20]:
rating = np.ones(len(test))
rating[test.user_id.nunique():] = 0
test["rating"] = rating

In [22]:
# numpy array
train_set = train.values
test_set = test[["user_id", "item_id", "rating"]].values

# to torch.Tensor
device = torch.device("cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.LongTensor(test_set).to(device)

In [23]:
n_dim = 10
n_user = ml.user_id.nunique()
n_item = ml.item_id.nunique()
X = csr_matrix(
    (np.ones(train_set.shape[0]), (train_set[:,0], train_set[:,1])),
    shape=[n_user, n_item]
)
U, V, ub, vb = svd_init(X, n_dim)

# Naive MF

In [24]:
device = torch.device("cpu")
lr = 1e-3
n_dim = 10
model = models.LogitMatrixFactorization(
    n_user, n_item, n_dim, max_norm=5,max_bias=3,
    user_embedding_init = torch.Tensor(U), 
    item_embedding_init = torch.Tensor(V.T),
    user_bias_init = torch.Tensor(ub), 
    item_bias_init = torch.Tensor(vb)
).to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.LogitPairwiseLoss().to(device)
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device,n_neg_samples=5, batch_size=1024)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(torch.LongTensor(test_set).to(device), score_function_dict, ks=[3])
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler)


In [25]:
trainer.fit(n_batch=50, n_epoch=15, valid_evaluator = evaluator, valid_per_epoch=5)

100%|██████████| 31360/31360 [13:12<00:00, 39.56it/s]


In [26]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,epoch,loss
0,0.999437,0.999697,0.004798,0,NaN
0,0.999398,0.999588,0.004797,5,395.838439
0,0.999388,0.999639,0.004790,10,202.204721
0,0.999424,0.999684,0.004786,15,119.902524


In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Default

In [30]:
lr = 1e-3
n_dim = 10
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.SumTripletLoss(margin=1).to(device)
sampler = samplers.BaseSampler(train_set.to(device), n_user, n_item, device=device, strict_negative=False)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(test_set.to(device), score_function_dict, ks=[3,5])
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler)


In [31]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 31360/31360 [03:23<00:00, 154.13it/s]


In [32]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.998453,0.998549,0.004692,0.998452,0.998568,0.007820,0,NaN
0,0.998408,0.998459,0.004689,0.998437,0.998527,0.007818,10,0.593670
0,0.998436,0.998504,0.004691,0.998438,0.998508,0.007817,20,0.546273


# Strict Negative

In [33]:
lr = 1e-3
n_dim = 10
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.SumTripletLoss(margin=1).to(device)
sampler = samplers.BaseSampler(train_set.to(device), n_user, n_item, device=device, strict_negative=True)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(test_set.to(device), score_function_dict, ks=[3,5])
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler)

In [34]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 31360/31360 [03:23<00:00, 153.97it/s]


In [35]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.998415,0.998475,0.004691,0.998420,0.998509,0.007818,0,NaN
0,0.998449,0.998517,0.004696,0.998458,0.998571,0.007826,10,0.575122
0,0.998496,0.998576,0.004696,0.998501,0.998609,0.007826,20,0.528670


# Two Stage

In [36]:
item_count = train.groupby("item_id")["user_id"].count()
count_index = np.array(item_count.index)
neg_weight = np.zeros(n_item)
neg_weight[count_index] = item_count ** 0.1

In [37]:
lr = 1e-3
n_dim = 10
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

regs = [regularizers.GlobalOrthogonalRegularizer(weight=1e-3)]
criterion = losses.MinTripletLoss(margin=1, regularizers=regs).to(device)
sampler = samplers.TwoStageSampler(
    train_set.to(device), n_user, n_item, 
    neg_weight=neg_weight, n_neg_samples=5,
    device=device, strict_negative=False
)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(test_set.to(device), score_function_dict, ks=[3,5])
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler)

In [38]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 31360/31360 [03:24<00:00, 153.61it/s]


In [39]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.998436,0.998483,0.004692,0.998423,0.998498,0.007819,0,NaN
0,0.998501,0.998597,0.004695,0.998501,0.998603,0.007823,10,1.117649
0,0.998552,0.998658,0.004698,0.998565,0.998664,0.007829,20,1.053914


In [ ]:
trainer.